# Combined Notebook for Canvas Group# 53

## Walkability Analysis by Justin Tometich:

In [5]:
import pandas as pd
import geopandas
import numpy as np

sidewalk_ratios_geoid = pd.read_csv("https://data.wprdc.org/dataset/853a077d-0a31-4292-8a1d-5d60b530169b/resource/b90ccee1-c0aa-43b9-93e2-8a25e690c393/download/sidewalkstreetratioupload.csv")
walk_scores_2014_data = pd.read_csv("https://data.wprdc.org/dataset/4d3d4324-b32a-4519-b56f-cee2340057bf/resource/682b1df1-a63b-4413-9362-ba077af63baa/download/walkscorect.xls-walk-score-by-ct.csv")
census_neighborhoods_2010 = geopandas.read_file("https://data.wprdc.org/dataset/4699bc46-8a74-44ed-9bef-f22dd595a964/resource/e51e4102-5aa1-4ef7-9f08-76fded66cb58/download/pittsburghpa2010-census-tracts.geojson")

census_neighborhoods_2010["geoid10"] = census_neighborhoods_2010["geoid10"].astype(int)
sidewalk_ratios_geoid["GEOID"] = sidewalk_ratios_geoid["GEOID"].astype(int)
sidewalk_ratios_with_neighborhood = census_neighborhoods_2010.merge(sidewalk_ratios_geoid, how = "left", left_on = "geoid10", right_on = "GEOID")

census_neighborhoods_2010 = geopandas.read_file("https://data.wprdc.org/dataset/4699bc46-8a74-44ed-9bef-f22dd595a964/resource/e51e4102-5aa1-4ef7-9f08-76fded66cb58/download/pittsburghpa2010-census-tracts.geojson")
walk_scores_2014_data["Census Tract"] = walk_scores_2014_data["Census Tract"].astype(int)
census_neighborhoods_2010["tractce10"] = census_neighborhoods_2010["tractce10"].astype(int)
walk_scores_with_neighborhood = census_neighborhoods_2010.merge(walk_scores_2014_data, how = "left", left_on = "tractce10", right_on = "Census Tract")


sidewalk_ratios_with_neighborhood["geoid10"] = sidewalk_ratios_with_neighborhood["geoid10"].astype(int)
walk_scores_with_neighborhood["geoid10"] = walk_scores_with_neighborhood["geoid10"].astype(int)
sidewalks_and_walk_scores_combined = walk_scores_with_neighborhood.merge(sidewalk_ratios_with_neighborhood[["hood", "Ratio", "geoid10"]], how = "left", left_on = "geoid10", right_on = "geoid10")

for index, row in sidewalks_and_walk_scores_combined.iterrows():
    # print(row)
    if row.loc["Ratio"] < 2:
        sidewalks_and_walk_scores_combined.at[index, "Walk Score"] = None

sidewalks_and_walk_scores_combined = geopandas.GeoDataFrame(sidewalks_and_walk_scores_combined)
sidewalks_and_walk_scores_combined.plot(column = "Walk Score", legend = True, legend_kwds={'label': "Walk Score"}, figsize=(15,10), edgecolor="black", missing_kwds={"color": "lightgrey"}).set_axis_off()

ModuleNotFoundError: No module named 'pandas'

In [ ]:
sidewalks_and_walk_scores_combined[["hood_x", "Ratio", "Walk Score"]].sort_values("Walk Score", ascending=False).head(10)

,hood_x,Ratio,Walk Score
89,Shadyside,2.778579,88.0
99,Squirrel Hill South,2.719089,88.0
80,Shadyside,2.695575,88.0
124,East Liberty,2.116163,86.0
35,Bloomfield,2.057364,86.0
78,Bloomfield,2.081824,86.0
39,Bloomfield,2.265926,82.0
66,North Oakland,4.045073,82.0
77,Bloomfield,2.700800,82.0
31,Squirrel Hill South,2.560622,80.0


### Walkability Analysis
Walkability was measured by two metrics from two different datasets: one about an already compiled Walk Score determined by combining distance from amenities, population density, road intersection density, etc. as well as another dataset specifically about the ratio of sidewalk length to road length. A sidewalk:road ratio of 2 indicates that, on average, each road will have sidewalks on both sides of the road. I then took Walk Score as my primary metric and filtered out neighborhoods with sidewalk:road ratios of less than 2 to determine which neighborhoods are most accessible by foot. Notably, downtown scores well in terms of Walk Score, but, likely because of freeways, alleyways, and other pedestrian-unfriendly roads, it falls below the ratio of 2 threshold. Thus, while downtown has the greatest density of amenities to bolster its walk score, it is discounted in this analysis. With the sidewalk ratio threshold taken into consideration, the top neighborhoods are Bloomfield, Shadyside, Squirrel Hill South, and East Liberty with Shadyside and Squirrel Hill South having similar walk scores but higher sidewalk ratios. Ratios greater than 2 indicate that more sidewalk exists than is needed to cover both directions of all roads in the neighborhood, which could be explained by pedestrian-only paths or sidewalks that cover the ends of cul-de-sacs (which, by having less car traffic, are a boon to pedestrians if they go to one). Also, the walk scores of these neighborhoods are in the mid- to upper-80s, which is only somewhat lower than the 100 score given to downtown.

## Best Bus Stops by Lucy Pry

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('bus.csv')
dropped = data.drop(columns=['STOP_ID','ROUTE','STOP_ROUTE','CLEVER_ID','ALL_ROUTES','STOP_TYPE','1806_ON','1806_OFF','1809_ON','1809_OFF','1811_ON','1811_OFF','1903_ON','1903_OFF','1906_ON','1906_OFF'])

busNA = dropped.dropna()
query_mask = busNA['CURRENT_STOP'].str.contains("Yes")
busBus = busNA[query_mask]
shelterFilter = busBus.drop(busBus.loc[busBus['SHELTER']=='No Shelter'].index)
avgOn = shelterFilter.query('FY19_AVG_ON >= 100')
bestOfTheBest = avgOn.query('FY19_AVG_OFF >= 100')

neighborhoods = pd.read_csv('Neighborhoods.csv')
neighborhoods['Bus Stops'].plot(kind = 'bar', x = "Neighborhoods", y = "Bus Stops")

### Bus Stop Analysis

The dataset I used was from 2019, because that was the last normal year before the COIVD-19 pandemic, therefore making it the best metric for as the world is returning to normal. The best part of Pittsburgh for people who are relying on the Port Authority busing system for transportation was measured through whether they had shelter at the stop, whether it was still an active stop, and then based on how often it was used. Any bus stops that did not have shelter or was no longer an active stop was filtered out. Then, if the bus stop had less than 100 people on or less than 100 people off it was filtered out because due to the size of Pittsburgh, less than 100 people is not a lot for a month and due to the unreliable nature of lesser used Port Authority stops, it is better to measure the more often used stops as there is a higher likelyhood they will have busses that arrive on time. Ergo, Downtown Pittsburgh ended up being the best place to live for someone relying on the busing system as it had the most amount of 'best' bus stops, sitting at 21 stops.